<a href="https://colab.research.google.com/github/BabakBagheriGisour2/Final-Assignment.ipynb/blob/main/Kopie_von_SIC_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2
!pip install sumy
!pip install transformers
!pip install rouge_score
!pip install sentencepiece
!pip install torch
!pip install Google


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 112.2 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21692 sha256=e5a66bdee9fb96db2d2e025d339ac1d5a485c35b635a8cae08209c3f3a910142
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=aaaac84b2cae247718a9fefa025f67779ac8b7896a9fbec1b775110bfd70c0f9
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-non

In [ ]:

import shutil
import os
from google.colab import drive

# اتصال به Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


**تبدیل پی دی اف به جیسون لاین**

In [ ]:
import os
import json
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
import fitz  # PyMuPDF
from tqdm import tqdm

# مسیرهای ورودی و خروجی
input_folder = '/content/'
output_folder = '/content/output'

# اطمینان حاصل می‌کنیم که پوشه خروجی وجود دارد
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# بارگذاری مدل و توکنایزر برای خلاصه‌سازی با facebook/bart-large-cnn
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

# بررسی اینکه آیا GPU موجود است یا خیر
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# تابعی برای استخراج متن از PDF و ایجاد خلاصه با BART
def extract_text_and_summary(pdf_path):
    # باز کردن فایل PDF
    doc = fitz.open(pdf_path)

    # لیست برای ذخیره داده‌ها
    data = []

    # پردازش هر صفحه
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text = page.get_text("text")  # استخراج متن صفحه

        # خلاصه‌سازی متن با استفاده از مدل BART
        inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True, padding=True)
        inputs = inputs.to(device)
        summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=150, early_stopping=True)

        # تبدیل IDهای خلاصه به متن
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        # اضافه کردن داده‌ها به لیست
        data.append({
            "text": text,
            "zusammenfassen": summary
        })

    return data

# خواندن فایل‌های PDF از پوشه ورودی و ذخیره آنها در پوشه خروجی
for filename in tqdm(os.listdir(input_folder)):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(input_folder, filename)

        # استخراج متن و خلاصه برای فایل PDF
        data = extract_text_and_summary(pdf_path)

        # نام فایل خروجی JSONL
        output_filename = os.path.splitext(filename)[0] + '.jsonl'
        output_path = os.path.join(output_folder, output_filename)

        # ذخیره داده‌ها در فایل JSONL
        with open(output_path, 'w', encoding='utf-8') as jsonl_file:
            for item in data:
                jsonl_file.write(json.dumps(item, ensure_ascii=False) + '\n')

print("تمامی فایل‌ها به JSONL تبدیل شدند.")


**آموزش مدل**

In [ ]:
import os
import json
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
from google.colab import drive

# اتصال به گوگل درایو
drive.mount('/content/drive')

# مسیر پوشه حاوی فایل‌های JSONL
jsonl_folder = '/content/output/'

# بارگذاری داده‌های JSONL
def load_jsonl_data(jsonl_folder):
    data = []
    for filename in os.listdir(jsonl_folder):
        if filename.endswith('.jsonl'):
            file_path = os.path.join(jsonl_folder, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    entry = json.loads(line)
                    data.append({
                        'text': entry['text'],
                        'summary': entry['zusammenfassen']
                    })
    return data

# بارگذاری داده‌ها
data = load_jsonl_data(jsonl_folder)

# ایجاد دیتاست از داده‌ها
dataset = Dataset.from_dict({
    'text': [entry['text'] for entry in data],
    'summary': [entry['summary'] for entry in data]
})

# بارگذاری مدل و توکنایزر BART
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# توکنایز کردن داده‌ها
def preprocess_function(examples):
    inputs = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=1024)
    targets = tokenizer(examples['summary'], padding="max_length", truncation=True, max_length=150)
    inputs["labels"] = targets["input_ids"]
    return inputs

# پیش‌پردازش داده‌ها
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# تنظیمات آموزشی
training_args = TrainingArguments(
    output_dir='./results',          # خروجی مدل
    evaluation_strategy="epoch",     # ارزیابی در هر دوره
    learning_rate=5e-5,              # نرخ یادگیری
    per_device_train_batch_size=4,   # اندازه بچ هر دستگاه
    per_device_eval_batch_size=8,    # اندازه بچ برای ارزیابی
    num_train_epochs=3,              # تعداد دوره‌های آموزش
    weight_decay=0.01,               # کاهش وزن
    logging_dir='./logs',            # مسیر لاگ‌ها
    logging_steps=10,                # فواصل زمانی لاگ‌گذاری
)

# استفاده از Trainer برای آموزش مدل
trainer = Trainer(
    model=model,                       # مدل BART
    args=training_args,                # تنظیمات آموزشی
    train_dataset=tokenized_dataset,   # داده‌های آموزشی
    eval_dataset=tokenized_dataset     # داده‌های ارزیابی
)

# آموزش مدل
trainer.train()

# ذخیره مدل در گوگل درایو
drive_model_path = '/content/drive/MyDrive/final_model'  # مسیر ذخیره مدل در گوگل درایو
model.save_pretrained(drive_model_path)
tokenizer.save_pretrained(drive_model_path)

print("Training complete and model saved to Google Drive.")


**ارزیابی مدل**

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer
from datasets import load_metric
import json
import matplotlib.pyplot as plt
from google.colab import drive

# اتصال به گوگل درایو و بارگذاری مدل و توکنایزر از درایو
drive.mount('/content/drive')

# مسیر مدل ذخیره‌شده در گوگل درایو
drive_model_path = '/content/drive/MyDrive/final_model'

# بارگذاری مدل و توکنایزر از گوگل درایو (مدل آموزش‌دیده)
model = BartForConditionalGeneration.from_pretrained(drive_model_path)
tokenizer = BartTokenizer.from_pretrained(drive_model_path)

# مسیر فایل JSON Line داده‌های اعتبارسنجی
validation_file_path = "validation_data.jsonl"

# بارگذاری معیار BLEU برای ارزیابی
bleu_metric = load_metric("bleu")

# لیست برای نگهداری ورودی‌ها و خروجی‌های واقعی و پیش‌بینی‌شده
true_texts = []
predicted_texts = []
bleu_scores_per_sample = []

# خواندن داده‌ها از فایل JSON Line و انجام پیش‌بینی مدل
with open(validation_file_path, 'r') as file:
    for i, line in enumerate(file):
        data = json.loads(line)

        # فرض می‌کنیم که ورودی‌ها در "input_text" و خروجی‌های واقعی در "target_text" هستند
        input_text = data["input_text"]
        target_text = data["target_text"]

        # توکنایز کردن ورودی و تولید پاسخ با استفاده از مدل آموزش‌دیده BART
        inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)
        output_ids = model.generate(inputs['input_ids'], max_length=150, num_beams=4, early_stopping=True)
        predicted_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # افزودن خروجی‌های واقعی و پیش‌بینی‌شده به لیست‌ها
        true_texts.append([target_text.split()])  # BLEU نیاز به آرایه‌های توکنیزه شده دارد
        predicted_texts.append(predicted_text.split())

        # محاسبه BLEU برای هر نمونه و اضافه کردن آن به لیست
        sample_bleu_score = bleu_metric.compute(predictions=[predicted_text.split()], references=[[target_text.split()]])["bleu"]
        bleu_scores_per_sample.append(sample_bleu_score)

        # چاپ اطلاعات هر نمونه
        print(f"Sample {i+1}")
        print("Input Text: ", input_text)
        print("Target Text: ", target_text)
        print("Predicted Text: ", predicted_text)
        print("BLEU Score for this sample: ", sample_bleu_score)
        print("-" * 50)

# محاسبه امتیاز BLEU کلی برای کل داده‌ها
overall_bleu_score = bleu_metric.compute(predictions=predicted_texts, references=true_texts)

# نمایش BLEU کلی
print("\nOverall BLEU Score:", overall_bleu_score["bleu"])

# رسم نمودار BLEU برای نمونه‌ها
plt.figure(figsize=(10, 6))
plt.plot(bleu_scores_per_sample, marker='o', linestyle='-', color='b')
plt.title("BLEU Score per Sample")
plt.xlabel("Sample Number")
plt.ylabel("BLEU Score")
plt.show()


**تست مدل**

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer
from datasets import load_metric
import json
import matplotlib.pyplot as plt

drive.mount('/content/drive')

# مسیر مدل ذخیره‌شده در گوگل درایو
drive_model_path = '/content/drive/MyDrive/final_model'
model = BartForConditionalGeneration.from_pretrained(drive_model_path)
tokenizer = BartTokenizer.from_pretrained(drive_model_path)

# مسیر فایل JSON Line داده‌های تست
test_file_path = "test_data.jsonl"

# بارگذاری معیار BLEU برای ارزیابی
bleu_metric = load_metric("bleu")

# لیست برای نگهداری ورودی‌ها و خروجی‌های واقعی و پیش‌بینی‌شده
true_texts = []
predicted_texts = []
bleu_scores_per_sample = []

# خواندن داده‌ها از فایل JSON Line و انجام پیش‌بینی مدل
with open(test_file_path, 'r') as file:
    for i, line in enumerate(file):
        data = json.loads(line)

        # فرض می‌کنیم که ورودی‌ها در "input_text" و خروجی‌های واقعی در "target_text" هستند
        input_text = data["input_text"]
        target_text = data["target_text"]

        # توکنایز کردن ورودی و تولید پاسخ با استفاده از مدل BART
        inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)
        output_ids = model.generate(inputs['input_ids'], max_length=150, num_beams=4, early_stopping=True)
        predicted_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # افزودن خروجی‌های واقعی و پیش‌بینی‌شده به لیست‌ها
        true_texts.append([target_text.split()])  # BLEU نیاز به آرایه‌های توکنیزه شده دارد
        predicted_texts.append(predicted_text.split())

        # محاسبه BLEU برای هر نمونه و اضافه کردن آن به لیست
        sample_bleu_score = bleu_metric.compute(predictions=[predicted_text.split()], references=[[target_text.split()]])["bleu"]
        bleu_scores_per_sample.append(sample_bleu_score)

        # چاپ اطلاعات هر نمونه
        print(f"Sample {i+1}")
        print("Input Text: ", input_text)
        print("Target Text: ", target_text)
        print("Predicted Text: ", predicted_text)
        print("BLEU Score for this sample: ", sample_bleu_score)
        print("-" * 50)

# محاسبه امتیاز BLEU کلی برای کل داده‌ها
overall_bleu_score = bleu_metric.compute(predictions=predicted_texts, references=true_texts)

# نمایش BLEU کلی
print("\nOverall BLEU Score:", overall_bleu_score["bleu"])

# رسم نمودار BLEU برای نمونه‌ها
plt.figure(figsize=(12, 6))
plt.plot(bleu_scores_per_sample, marker='o', linestyle='-', color='b')
plt.title("BLEU Score per Sample")
plt.xlabel("Sample Number")
plt.ylabel("BLEU Score")
plt.grid(True)
plt.show()

# رسم هیستوگرام برای توزیع امتیازات BLEU
plt.figure(figsize=(10, 5))
plt.hist(bleu_scores_per_sample, bins=20, color='purple', alpha=0.7)
plt.title("BLEU Score Distribution")
plt.xlabel("BLEU Score")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()
